<a href="https://colab.research.google.com/github/MounishPorandla/Adoption/blob/master/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
od.download("https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mounishm2204
Your Kaggle Key: ··········


100%|██████████| 8.89M/8.89M [00:00<00:00, 123MB/s]

In [7]:
df=pd.read_csv("/content/tmdb-movie-metadata/tmdb_5000_movies.csv")
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [8]:
def get_director(list):
    l=[]
    for i in eval(list):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l


In [29]:
frame=pd.read_csv("/content/tmdb-movie-metadata/tmdb_5000_credits.csv")
frame.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [31]:
frame['director']=frame['crew'].apply(get_director)

In [32]:
frame.head()

,movie_id,title,cast,crew,director
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",[Gore Verbinski]
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",[Christopher Nolan]
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",[Andrew Stanton]


In [34]:
df.head()
di=df.dropna()
data1=df['original_title']
data1=pd.concat([dj,df['overview']],axis='columns')
data1

,original_title,overview,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca...","John Carter is a war-weary, former military ca..."
...,...,...,...
4798,El Mariachi,El Mariachi just wants to play his guitar and ...,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...,When ambitious New York attorney Sam is sent t...


In [37]:
tfidf=TfidfVectorizer(stop_words="english")
tr = tfidf.fit_transform(df['overview'].apply(lambda a: np.str_(a)))

In [38]:
tr

<4803x20979 sparse matrix of type '<class 'numpy.float64'>'
	with 125843 stored elements in Compressed Sparse Row format>

In [39]:
cos_sim=linear_kernel(tr,tr)

In [40]:
movie_indices=pd.Series(df.index, index=df['original_title']).drop_duplicates()


In [41]:
movie_indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [18]:
def movie_recomendations(title, cos_sim=cos_sim):
  idx=indices[title]
  scores=pd.Series(cos_sim[idx]).sort_values(ascending=False)
  topsc=list(scores.iloc[1:11].index)
  for i in topsc:
    print(df['original_title'][i])
    print(scores[i])
  

In [20]:
movie_recomendations("Avatar")

Apollo 18
0.18681001056212054
The American
0.17015105119162663
The Matrix
0.13230696630317224
Obitaemyy Ostrov
0.12393264590871879
Tears of the Sun
0.12248959164893028
Hanna
0.10469903037615635
The Adventures of Pluto Nash
0.10091388184065117
Semi-Pro
0.09799623121705058
Supernova
0.09447938436890335
Blood and Chocolate
0.09313854469530522


In [21]:
b=tfidf.fit_transform(df['overview'].apply(lambda b: np.str_(b)))

In [22]:
cos_sims=cosine_similarity(b,b)

In [24]:
def movie_recomendation(title, cos_sims=cos_sims):
  idx=indices[title]
  scores=pd.Series(cos_sims[idx]).sort_values(ascending=False)
  topsc=list(scores.iloc[1:11].index)
  for i in topsc:
    print(df['original_title'][i])
    print(scores[i])

In [25]:
movie_recomendation("Avatar")

Apollo 18
0.18681001056212054
The American
0.17015105119162666
The Matrix
0.13230696630317224
Obitaemyy Ostrov
0.12393264590871879
Tears of the Sun
0.12248959164893031
Hanna
0.10469903037615635
The Adventures of Pluto Nash
0.10091388184065118
Semi-Pro
0.09799623121705059
Supernova
0.09447938436890338
Blood and Chocolate
0.09313854469530522


In [26]:
c=tfidf.fit_transform(dj['overview'].apply(lambda c: np.str_(c)))
d=tfidf.fit_transform(dj['original_title'].apply(lambda d: np.str_(d)))
e=tfidf.fit_transform(af['director'].apply(lambda d: np.str_(d)))
cos_simd=cosine_similarity(c,c)
cos_sime=cosine_similarity(d,d)
cos_simf=cosine_similarity(e,e)

In [42]:
def movie_recomends(title, cos_simd=cos_simd):
  idx=indices[title]
  scores=pd.Series(cos_simd[idx])
  scoress=pd.Series(cos_sime[idx])
  scoresss=pd.Series(cos_simf[idx])
  scoresum=scores+scoress+scoresss
  scoresum=scoresum.sort_values(ascending=False)
  topsc=list(scoresum.iloc[1:11].index)
  for i in topsc:
    print(df['original_title'][i])
    print(scoresum[i])

In [43]:
movie_recomends("The Godfather")

The Godfather: Part II
2.20538660927713
The Godfather: Part III
1.9140985282987308
The Last Godfather
1.026752118950128
New York Stories
1.0216868595942925
Twixt
1.006036159787692
The Rainmaker
1.0
Dracula
1.0
Peggy Sue Got Married
1.0
The Cotton Club
1.0
Apocalypse Now
1.0
